<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/Assignment_1_Langgraph_based_Automotive_Telemetry_System_HCLAICafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain langgraph langchain-openai faiss-cpu

In [16]:
from google.colab import userdata
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"] = "a8296d0e-0312-429f-9fd7-729c4fdc4b12"
os.environ["AZURE_OPENAI_ENDPOINT"] = ("https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview")
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"

# Get the API key from environment variables
#openai_api_key = userdata.get("OPENAI_API_KEY")
#import os
#os.environ["OPENAI_API_KEY"] = openai_api_key

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    temperature=0
)

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/ada/embeddings?api-version=2023-05-15",
    api_key="a8296d0e-0312-429f-9fd7-729c4fdc4b12"
)

In [17]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
    azure_deployment="gpt-4.1",
    temperature=0
)

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint="https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/deployments/ada/embeddings?api-version=2023-05-15",
    api_key="a8296d0e-0312-429f-9fd7-729c4fdc4b12"
)

In [18]:
!pip install langchain_community

In [19]:
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

#from langchain.schema import Document

docs = [
    Document(page_content="High engine temperature may indicate coolant leak or radiator failure."),
    Document(page_content="Low battery voltage often suggests battery degradation."),
    Document(page_content="Brake pressure loss is usually caused by hydraulic fluid leakage."),
    Document(page_content="Brake system faults are safety critical and require immobilization.")
]

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [20]:
from typing import TypedDict, List, Dict

class VehicleState(TypedDict):
    vehicle_id: str
    telemetry: Dict[str, float]
    anomaly: str
    retrieved_docs: List[str]
    diagnosis: str
    decision: str

In [21]:
def telemetry_node(state: VehicleState):
    return state

In [22]:
def anomaly_node(state: VehicleState):
    t = state["telemetry"]

    if t["engine_temp"] > 110:
        state["anomaly"] = "High engine temperature"
    elif t["battery_voltage"] < 11.5:
        state["anomaly"] = "Low battery voltage"
    elif t["brake_pressure"] < 20:
        state["anomaly"] = "Brake pressure loss"
    else:
        state["anomaly"] = "Normal"

    return state

In [23]:
def retrieval_node(state: VehicleState):
    docs = retriever.invoke(state["anomaly"])
    state["retrieved_docs"] = [d.page_content for d in docs]
    return state

In [24]:
def diagnosis_node(state: VehicleState):
    context = "\n".join(state["retrieved_docs"])

    prompt = f"""
    You are an automotive diagnostics expert.

    Telemetry:
    {state["telemetry"]}

    Detected anomaly:
    {state["anomaly"]}

    Relevant knowledge:
    {context}

    Identify the most likely root cause.
    """

    response = llm.invoke(prompt)
    state["diagnosis"] = response.content
    return state

In [25]:
def decision_node(state: VehicleState):
    prompt = f"""
    You are a fleet safety AI.

    Diagnosis:
    {state["diagnosis"]}

    Decide the safest action.
    Constraints:
    - Human safety first
    - Vehicle protection second
    - Cost last

    Respond with ONE action.
    """

    response = llm.invoke(prompt)
    state["decision"] = response.content
    return state

In [26]:
def report_node(state: VehicleState):
    print("\n===== VEHICLE INCIDENT REPORT =====")
    print("Vehicle ID:", state["vehicle_id"])
    print("Telemetry:", state["telemetry"])
    print("Anomaly:", state["anomaly"])
    print("Diagnosis:", state["diagnosis"])
    print("Decision:", state["decision"])
    print("=================================\n")

    return state

In [27]:
from langgraph.graph import StateGraph, END

graph = StateGraph(VehicleState)

graph.add_node("telemetry", telemetry_node)
graph.add_node("anomaly", anomaly_node)
graph.add_node("retrieve", retrieval_node)
graph.add_node("diagnose", diagnosis_node)
graph.add_node("decide", decision_node)
graph.add_node("report", report_node)

graph.set_entry_point("telemetry")

graph.add_edge("telemetry", "anomaly")
graph.add_edge("anomaly", "retrieve")
graph.add_edge("retrieve", "diagnose")
graph.add_edge("diagnose", "decide")
graph.add_edge("decide", "report")
graph.add_edge("report", END)

app = graph.compile()

In [28]:
initial_state = {
    "vehicle_id": "CAR-9001",
    "telemetry": {
        "engine_temp": 128,
        "battery_voltage": 12.1,
        "brake_pressure": 30
    },
    "anomaly": "",
    "retrieved_docs": [],
    "diagnosis": "",
    "decision": ""
}

app.invoke(initial_state)


===== VEHICLE INCIDENT REPORT =====
Vehicle ID: CAR-9001
Telemetry: {'engine_temp': 128, 'battery_voltage': 12.1, 'brake_pressure': 30}
Anomaly: High engine temperature
Diagnosis: Based on the telemetry:

- Engine temperature: 128°C (abnormally high)
- Battery voltage: 12.1V (slightly low)
- Brake pressure: 30 (assumed normal)

Detected anomaly: High engine temperature

Relevant knowledge:
- High engine temperature may indicate coolant leak or radiator failure.
- Low battery voltage often suggests battery degradation.

**Analysis:**
The primary anomaly is the high engine temperature. The most common causes are:
- Coolant leak (insufficient coolant to dissipate heat)
- Radiator failure (ineffective cooling system)

The battery voltage is slightly low but not critically so, and is less likely to directly cause overheating unless the alternator is failing, which would also affect other electrical systems.

**Conclusion:**
The most likely root cause is a coolant leak or radiator failure, 

{'vehicle_id': 'CAR-9001',
 'telemetry': {'engine_temp': 128,
  'battery_voltage': 12.1,
  'brake_pressure': 30},
 'anomaly': 'High engine temperature',
 'retrieved_docs': ['High engine temperature may indicate coolant leak or radiator failure.',
  'Low battery voltage often suggests battery degradation.'],
 'diagnosis': 'Based on the telemetry:\n\n- Engine temperature: 128°C (abnormally high)\n- Battery voltage: 12.1V (slightly low)\n- Brake pressure: 30 (assumed normal)\n\nDetected anomaly: High engine temperature\n\nRelevant knowledge:\n- High engine temperature may indicate coolant leak or radiator failure.\n- Low battery voltage often suggests battery degradation.\n\n**Analysis:**\nThe primary anomaly is the high engine temperature. The most common causes are:\n- Coolant leak (insufficient coolant to dissipate heat)\n- Radiator failure (ineffective cooling system)\n\nThe battery voltage is slightly low but not critically so, and is less likely to directly cause overheating unless 